In [1]:
import pennylane as qml
import numpy as np
import scipy as sci
from lib import MPS,init_spinup_MPS,vn_entropy
import time

In [2]:
nWires = 6

In [3]:
device = qml.device("lightning.qubit",wires=nWires)

@qml.qnode(device)
def pennylane_heating(N:int,gates:list,wire_indices:list):
    # the gates from which we choose
    gate_set = {
        "CNOT":qml.CNOT,
        "T":qml.T,
        "H":qml.Hadamard,
        "X":qml.X,
        "S":qml.S
    }

    # rotating each qubit to create the initial state from the paper
    for iWire,phi in enumerate(np.linspace(start=1,stop=np.pi,num=nWires)):#enumerate(np.random.uniform(low=0,high=np.pi,size=(nWires,))):
        qml.RY(phi=phi,wires=iWire)

    for iStep,gate in enumerate(gates):
        gate_set[gate](wires=wire_indices[iStep])

    return qml.state()

In [4]:
def MPS_heating(nWires:int,nSteps:int,gates:list,wire_indices:list) -> MPS:
    state = init_spinup_MPS(L=nWires)

    # rotating each qubit to create the initial state from the paper
    RY = lambda phi: np.array([[np.cos(phi/2),(-1)*np.sin(phi/2)],[np.sin(phi/2),np.cos(phi/2)]],dtype=np.complex128)
    for iWire,theta in enumerate(np.linspace(start=1,stop=np.pi,num=nWires)):#enumerate(np.random.uniform(low=0,high=np.pi,size=(nWires,))):
        state.apply_operator(RY(theta),iWire)

    H = np.array([[1,1],[1,-1]],dtype=np.complex128) / np.sqrt(2)
    X = np.array([[0,1],[1,0]],dtype=np.complex128)
    S = np.array([[1,0],[0,1j]],dtype=np.complex128)
    T = np.array([[1,0],[0,np.exp(1j * np.pi / 4)]],dtype=np.complex128)
    CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]],dtype=np.complex128)
    gate_set = {
        "CNOT":CNOT,
        "T":T,
        "H":H,
        "X":X,
        "S":S
    }

    norm = np.linalg.norm(state.uncompress())
    for iStep,gate in enumerate(gates):
        # state.check_normalization()
        wires = wire_indices[iStep]
        # print(gate + " on wires ",wire_indices[iStep])
        state.apply_operator(gate_set[gate],*wires)

    return state

In [5]:
def statevec_heating(nWires:int,nSteps:int,gates:list,wire_indices:list) -> np.ndarray:
    """
    Entanglement heating of a random product state, using the gates in `gate_set`. Algorithm from "Irreversibility and Entanglement
    Spectrum Statistics in Quantum Circuits", Shaffer et Al, 2014.
    
    Returns the resulting state and the entanglement entropy at every bond for every iteration.
    """
    # initial state
    state = np.zeros(shape=(2**nWires,),dtype=np.complex128)
    state[0] = 1
    state = np.reshape(state,newshape=[2 for iWire in range(nWires)])

    # rotating each qubit to create the initial state from the paper
    RY = lambda phi: np.array([[np.cos(phi/2),(-1)*np.sin(phi/2)],[np.sin(phi/2),np.cos(phi/2)]],dtype=np.complex128)
    for iWire,theta in enumerate(np.linspace(start=1,stop=np.pi,num=nWires)):#enumerate(np.random.uniform(low=0,high=np.pi,size=(nWires,))):
        state = np.tensordot(RY(theta),state,axes=(1,iWire))
        indices = np.concatenate((
            np.arange(iWire)+1,
            [0,],
            np.arange(iWire + 1,nWires)
        ))
        state = np.transpose(state,indices)

    H = np.array([[1,1],[1,-1]],dtype=np.complex128) / np.sqrt(2)
    X = np.array([[0,1],[1,0]],dtype=np.complex128)
    S = np.array([[1,0],[0,1j]],dtype=np.complex128)
    T = np.array([[1,0],[0,np.exp(1j * np.pi / 4)]],dtype=np.complex128)
    CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]],dtype=np.complex128)
    gate_set = {
        "CNOT":CNOT,
        "T":T,
        "H":H,
        "X":X,
        "S":S
    }

    for iStep,gate in enumerate(gates):
        wires = wire_indices[iStep]

        if len(wires) == 1:
            iWire = wires[0]
            # we got ourselves a single-qubit gate
            state = np.tensordot(gate_set[gate],state,axes=((1,),(iWire,)))
            indices = np.concatenate((
                np.arange(iWire)+1,
                [0,],
                np.arange(iWire + 1,nWires)
            ))
            state = np.transpose(state,indices)
        else:
            # we got ourselves a two-qubit gate
            i1,i2 = wires
            if i2 < i1:
                # since the indices are flipped, we also need to flip the gate
                SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]],dtype=np.complex128)
                op = SWAP @ gate_set[gate] @ SWAP
                i1,i2 = i2,i1
            else:
                op = gate_set[gate]
            op = np.reshape(op,(2,2,2,2))
            state = np.tensordot(op,state,axes=((2,3),(i1,i2)))
            indices = np.concatenate((
                np.arange(i1) + 2,
                [0,],
                np.arange(i2 - i1 - 1) + i1 + 2,
                [1,],
                np.arange(nWires-i2-1) + i2 + 1
            ))
            state = np.transpose(state,indices)
    
    return state.flatten()

In [6]:
gate_set = ("CNOT","H","T")
num_wires = {"CNOT":2,"H":1,"T":1,"X":1,"S":1}
nSteps = 10

gates = [gate_set[i] for i in np.random.randint(low=0,high=len(gate_set),size=(nSteps))]
wire_indices = [tuple(np.random.choice(a=nWires,size=(num_wires[gate],),replace=False)) for gate in gates]

#gates = ["CNOT","CNOT","CNOT","CNOT",]
#wire_indices = [(1,0),(5,1),(0,3),(1,5)]

In [7]:
t1 = time.time()
state_pennylane = pennylane_heating(nSteps,gates,wire_indices)
t2 = time.time()
MPS_custom = MPS_heating(nWires,nSteps,gates,wire_indices)
t3 = time.time()
state_psi = statevec_heating(nWires,nSteps,gates,wire_indices)
t4 = time.time()

state_MPS = MPS_custom.uncompress()

print("norm of pennylane:  ",np.einsum("i,i->",state_pennylane,state_pennylane.conj()))
print("norm of MPS custom: ",np.einsum("i,i->",state_MPS,state_MPS.conj()))
print("norm of psi custom: ",np.einsum("i,i->",state_psi,state_psi.conj()))

print("\nAre they close?")
print("   pennylane and MPS:",np.allclose(state_pennylane,state_MPS))
print("   pennylane and psi:",np.allclose(state_pennylane,state_psi))
print("   MPS       and psi:",np.allclose(state_psi,state_MPS))

print("\nHow long did it take?")
print("   pennylane: {:.5f}s".format(t2 - t1))
print("   MPS:       {:.5f}s".format(t3 - t2))
print("   psi:       {:.5f}s".format(t4 - t3))

Checking normalization.
  Norm of complete MPS = 1.000e+00
CNOT on wires  (2, 0)
  Psi not normalized! |psi[1,2]| = 1.414
Checking normalization.
  Norm of complete MPS = 7.071e-01
  psi[0,1] is not normalized! |psi[0,1]| = 7.071e-01
  psi[0,2] is not normalized! |psi[0,2]| = 7.071e-01
  psi[0,3] is not normalized! |psi[0,3]| = 7.071e-01
  psi[0,4] is not normalized! |psi[0,4]| = 7.071e-01
  psi[0,5] is not normalized! |psi[0,5]| = 7.071e-01
  psi[1,2] is not normalized! |psi[1,2]| = 7.071e-01
  psi[1,3] is not normalized! |psi[1,3]| = 7.071e-01
  psi[1,4] is not normalized! |psi[1,4]| = 7.071e-01
  psi[1,5] is not normalized! |psi[1,5]| = 7.071e-01
CNOT on wires  (1, 3)
Incoming psi is not normalized!
  Norm of complete MPS = 7.071e-01
  Psi not normalized! |psi[1,3]| = 0.707
  Psi not normalized! |psi[2,3]| = 1.414
Checking normalization.
  Norm of complete MPS = 5.000e-01
  psi[0,2] is not normalized! |psi[0,2]| = 5.000e-01
  psi[0,3] is not normalized! |psi[0,3]| = 5.000e-01
  psi[

In [8]:
rho_pennylane = np.einsum("i,j->ij",state_pennylane,state_pennylane.conj())
rho_MPS = np.einsum("i,j->ij",state_MPS,state_MPS.conj())
S_MPS = MPS_custom.entanglement_entropy()

indices = ()
S_pennylane = ()
for iWire in range(nWires-1):
    indices += (iWire,)
    S_pennylane += (qml.math.vn_entropy(rho_pennylane,indices,base=2),)
S_pennylane = np.array(S_pennylane)

In [9]:
print("Entanglement entropy differences:")
if np.allclose(S_pennylane,vn_entropy(state_pennylane)): print("  Function vn_entropy is correct.")
if np.allclose(S_pennylane,vn_entropy(state_MPS)): print("  MPS statevector gives the correct entropy.")
if np.allclose(S_pennylane,vn_entropy(state_psi)): print("  Statevector simulation gives the correct entropy.")
if np.allclose(S_pennylane,S_MPS):
    print("  MPS bond singular values give the correct entropy.")
else:
    print("  MPS bond singular values give the wrong entropy; max diff = {:.5e}; avg diff = {:.5e}".format(max(abs(S_pennylane - S_MPS)),abs(np.mean(S_pennylane)-np.mean(S_MPS))))

Entanglement entropy differences:
  Function vn_entropy is correct.


AssertionError: 